Define classes

In [1]:
import requests
from pprint import pprint
import os

class MyUrl:
    def __init__(self, url):
        self.url = url.rstrip('/')
    
    def get_current(self):
        return self.url
    
    
class Printer:
    def __init__(self, endpoint):
        self.endpoint = endpoint
        
    def print_keys(self, data):
        self.print_url(self.endpoint.get_current())
        
        print("KEYS")
        print('-' * 40)
        pprint(list(data.keys()))
        
    def print_data(self, data):
        self.print_url(self.endpoint.get_current())
        
        print("DATA")
        print('-' * 40)
        pprint(data)
        
    def print_url(self, url):
        print("CURRENT URL:", url)
        print('-' * 40)
        
            
class PxWeb:
    def fetch(self, url):
        response = requests.get(url)
        if response.status_code == 200:    
            return response.json()
        else:
            print('Error retrieving data')
            
    def post(self, url, data):
        response = requests.post(url, json=data)
        if response.status_code == 200:    
            return response.json()
        else:
            print('Error retrieving data')
            
class ObjectOfInterest:
    def __init__(self):
        self.data: None
        self.name: None
        self.code: None
        self.valueTexts: None
        self.values: None
        
    def get_keys(self):
        return list(self.data.keys())
    
    def get_data(self):
        return self.data
    
    def get_name(self):
        return self.name
    
    def get_code(self):
        return self.code
    
    def get_valueTexts(self):
        return self.valueTexts
    
    def get_values(self):
        return self.values
    
class ExtractedData:
    def __init__(self):
        self.data = None
        
    def set_data(self, data):
        self.data = data
        
    def get_data(self):
        return self.data
    
class FileHandler:
    def __init__(self, filename, type='json'):
        self.filename = filename
        self.type = type
        
    def write(self, data):
        with open(f'{self.filename}.{self.type}', 'w') as file:
            file.write(data)
            
    def read(self):
        with open(f'{self.filename}.{self.type}', 'r') as file:
            return file.read()
        
    def append(self, data):
        with open(f'{self.filename}.{self.type}', 'a') as file:
            file.write(data)
            
    def delete(self):
        if os.path.exists(self.filename):
            os.remove(self.filename)
        else:
            print("The file does not exist")
    
            
# Initialize the classes
endpoint = MyUrl(url='https://pxdata.stat.fi:443/PxWeb/api/v1/fi/Kuntien_avainluvut/uusin/142h.px/')

printer = Printer(endpoint=endpoint)

pw = PxWeb()

obi = ObjectOfInterest()

exd = ExtractedData()

ValuesFile = FileHandler(filename='values', type='json')


Build data object

In [2]:
data = pw.fetch(endpoint.get_current())

obi.data = data['variables'][1]
obi.name = obi.data['text']
obi.code = obi.data['code']
obi.valueTexts = obi.data['valueTexts']
obi.values = obi.data['values']

Extract two kinds of key-value paired objects:
    - valueTexts: values
    - values: valueTexts

In [3]:
valueTexts = obi.get_valueTexts()
values = obi.get_values()

# Check the length matches
if len(valueTexts) == len(values):
    
    # Map new key-value object
    new_obj = {}
    for i in range(len(valueTexts)):
        new_obj[values[i]] = valueTexts[i]
        
    exd.set_data(new_obj)
    
else:
    raise Exception("Lengths do not match")


Writing to file: valueTextsForValues.json

In [4]:
import json

ValuesFile.write(json.dumps(exd.get_data()))